# **Summarisation & Evasion Notebook (LB)**

# **1. Objective**

- Summarise banker answers into short, PRA relevant insights.
- Generate an evasion score to tag summaries.
- RAG pipeline to bring in relevant external documents (e.g. PRA risk definitions, regulatory news).
- Optional extension: validate flagged risks with external/regulatory news.

# **2. Set up Workspace**

In [ ]:
# Import libraries
# Core python
import os
import numpy as np
import pandas as pd
import re

# NLP & Summarisation
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
import spacy

# Evaluation
from rouge_score import rouge_scorer
import evaluate
from bert_score import score as bertscore 

# Retrieval
from sentence_transformers import SentenceTransformer 
import faiss
import chromadb
import langchain
import llama_index

# ML
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Visualisations
import matplotlib.pyplot as plt
import seaborn as sns 

# **3. Load the dataset**

In [14]:
# Load the dataset.
jpm_2025_df = pd.read_csv('../data/processed/jpm/all_jpm_2025.csv')

# View the data.
jpm_2025_df.head()

,question_number,answer_number,speaker_name,role,company,content,year,quarter,source_pdf
0,1,NaN,Ken Usdin,analyst,Autonomous Research,"Good morning, Jeremy. Wondering if you could s...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
1,1,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Sure, Ken. So I mean, at a high level, I would...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
2,2,NaN,Ken Usdin,analyst,Autonomous Research,Yeah. And just one question on the NII ex. Mar...,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
3,2,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Yeah, that's a good question, Ken. You're righ...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
4,2,2.0,Jamie Dimon,Chairman & Chief Executive Officer,JPMorganChase,In the curve basically.,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...


# **4. Preprocessing**

In [18]:
# View speaker roles.
jpm_2025_df['role'].unique()

array(['analyst', 'Chief Financial Officer',
       'Chairman & Chief Executive Officer',
       'And then some. Theres a lot of value added.', 'Okay'],
      dtype=object)

In [17]:
# View rows with invalid roles.
valid_roles = 'analyst', 'Chief Financial Officer', 'Chairman & Chief Executive Officer'
invalid_roles_df = jpm_2025_df[~jpm_2025_df['role'].isin(valid_roles)]

# Number of rows with invalid roles.
print('Number of rows:', invalid_roles_df.shape[0])

# View the rows.
invalid_roles_df.head()

Number of rows: 2


,question_number,answer_number,speaker_name,role,company,content,year,quarter,source_pdf
205,37,5.0,"Chief Financial Officer, JPMorganChase",And then some. Theres a lot of value added.,JPMorganChase,"Yeah. And obviously, I mean, we're not going t...",2025,Q2,data/raw/jpm/jpm-2q25-earnings-call-transcript...
209,38,3.0,"Chief Financial Officer, JPMorganChase",Okay,there you have it.,"But it's not like I thought it would do badly,...",2025,Q2,data/raw/jpm/jpm-2q25-earnings-call-transcript...


In [ ]:
# Input the correct role information.
jpm_2025_df.at[205, 'role'] = 'Chief Financial Officer'
jpm_2025_df.at[209, 'role'] = 'Chief Financial Officer'

# Verify the roles have been updated.
jpm_2025_df['role'].unique()

array(['analyst', 'Chief Financial Officer',
       'Chairman & Chief Executive Officer'], dtype=object)

In [22]:
# Define role mapping.
role_map = {
    'analyst': 'analyst',
    'Chief Financial Officer': 'banker',
    'Chairman & Chief Executive Officer': 'banker'
}

# Apply to dataset.
jpm_2025_df['role_normalised'] = jpm_2025_df['role'].map(role_map)

In [23]:
# View the dataset.
jpm_2025_df.head()

,question_number,answer_number,speaker_name,role,company,content,year,quarter,source_pdf,role_normalised
0,1,NaN,Ken Usdin,analyst,Autonomous Research,"Good morning, Jeremy. Wondering if you could s...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,analyst
1,1,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Sure, Ken. So I mean, at a high level, I would...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,banker
2,2,NaN,Ken Usdin,analyst,Autonomous Research,Yeah. And just one question on the NII ex. Mar...,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,analyst
3,2,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Yeah, that's a good question, Ken. You're righ...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,banker
4,2,2.0,Jamie Dimon,Chairman & Chief Executive Officer,JPMorganChase,In the curve basically.,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,banker


# **5. Summarisation**

## **5.1 Baseline**

In [26]:
# Filter data to banker answers only.
banker_answers = jpm_2025_df[jpm_2025_df['role_normalised'] == 'banker']['content'].tolist()
print(banker_answers[0][:200])

Sure, Ken. So I mean, at a high level, I would say that obviously, some of the salient news flow is quite recent. So, we've done some soundings and some checking both on the consumer side and on the w


In [ ]:
# Summarisation baseline (BART)
summariser = pipeline('summarization', model='facebook/bart-large-cnn')

sample_text = banker_answers[0]
summary = summariser(sample_text, max_length=80, min_length=30, do_sample=False)
print('Original:', sample_text[:400])
print('Summary:', summary[0]['summary_text'])

Device set to use mps:0


Original: Sure, Ken. So I mean, at a high level, I would say that obviously, some of the salient news flow is quite recent. So, we've done some soundings and some checking both on the consumer side and on the wholesale side. I think on the consumer side, the thing to check is the spending data. And to be honest, the main thing that we see there, what would appear to be a certain amount of frontloading of sp
Summary: The main thing that we see there, what would appear to be a certain amount of frontloading of spending ahead of people expecting price increases from tariffs. So ironically, that's actually somewhat supportive, all else equal. In terms of our corporate clients, obviously, they've been reacting to the changes in tariff policy.


In [31]:
# Prompt conditioning to make PRA relevant.
prompt = "Summarise this answer, focusing on risk, capital and evasion of detail: " + sample_text
summary = summariser(prompt, max_length=80, min_length=30)
print('Original:', sample_text[:400])
print('Summary:', summary[0]['summary_text'])

Original: Sure, Ken. So I mean, at a high level, I would say that obviously, some of the salient news flow is quite recent. So, we've done some soundings and some checking both on the consumer side and on the wholesale side. I think on the consumer side, the thing to check is the spending data. And to be honest, the main thing that we see there, what would appear to be a certain amount of frontloading of sp
Summary: Corporates are taking a wait-and-see approach to tariff policy. Some sectors are going to be much more exposed than others. Small business and smaller corporates are probably a little more challenged.


## **5.2 RAG**

Retrieve PRA risk categories to give greater PRA focus to summaries (local RAG loop).
- measure semantic similarity between answers and risks
- retrieve the top 2-3 most relevant risk categories
- prepend them to the summarisation prompt to make summaries PRA-aligned instead of just summarised answers

In [34]:
# Load PRA risk categories.
pra_risk_df = pd.read_csv('../data/RAG-resources/PRA_risk_categories.csv')
pra_risk_df['doc'] = pra_risk_df['category'] + ': ' + pra_risk_df['definition']

# View dataframe.
pra_risk_df

,category,definition,doc
0,Capital adequacy,"""Sufficiency of capital to absorb losses; CET1...","Capital adequacy: ""Sufficiency of capital to a..."
1,Liquidity,"""Ability to meet obligations as they fall due;...","Liquidity: ""Ability to meet obligations as the..."
2,Credit risk,"""Counterparty default/credit deterioration; pr...","Credit risk: ""Counterparty default/credit dete..."
3,Market risk,"""P&L volatility from market movements; VaR, se...","Market risk: ""P&L volatility from market movem..."
4,Operational risk,"""Loss from processes/people/systems; cyber, fr...","Operational risk: ""Loss from processes/people/..."
5,Conduct risk,"""Customer outcomes and market integrity; mis-s...","Conduct risk: ""Customer outcomes and market in..."
6,Governance,"""Board oversight, controls, risk culture, remu...","Governance: ""Board oversight, controls, risk c..."
7,Risk management,"""Risk frameworks, appetite, limits, aggregatio...","Risk management: ""Risk frameworks, appetite, l..."
8,Stress testing,"""Outcomes under stress scenarios; capital and ...","Stress testing: ""Outcomes under stress scenari..."


In [42]:
# Create embeddings of each doc (defintion + risk).
embed = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

pra_risk_embeddings = embed.encode(pra_risk_df['doc'].tolist(), normalize_embeddings=True).astype('float32')
index = faiss.IndexFlatIP(pra_risk_embeddings.shape[1])
index.add(pra_risk_embeddings)

- creates embeddings of doc
- adds PRA vectors to the index so you can query which categories are most similar to the answer

In [43]:
# Retrieve the most relevant risk categories + defintions 
def retrieve_pra_context(query_text: str, k: int = 3) -> pd.DataFrame:
    q = embed.encode([query_text], normalize_embeddings=True).astype('float32')
    scores, idxs = index.search(q, k)
    out = pra_risk_df.iloc[idxs[0]].copy()
    out['score'] = scores[0]
    return out 

Simple retriever:
- encodes the bankers answer into a vector, searches the index and then returns the top k categories with similarity scores

In [ ]:
# # Convert retrieved risk doc into a context block to prompt the model.
# def make_context_block(retrieved_df: pd.DataFrame) -> str:
#     lines = [f"- {r['category']}: {r['definition']}" for _, r in retrieved_df.iterrows()]
#     return '\n'.join(lines)

In [ ]:
# Convert retrieved risk doc into a pra keyword to prompt the model.
def pra_keywords(retrieved_df: pd.DataFrame, top_n: int = 3) -> str:
    return ', '.join(retrieved_df['category'].head(top_n).tolist())

In [ ]:
# Chunking the banker's answers to avoid truncation.
def split_into_chunks(text: str, max_chunk_size: int = 1800):

    # split text into sentences
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())

    # list to store chunks
    chunks = []
    current_chunk = ''
    for s in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:
            current_chunk = f'{current_chunk} {sentence}'.strip()
        else:
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = sentence 
    
    if current_chunk: 
        chunks.append(current_chunk)

    return chunks

In [ ]:
# # Function to create a PRA risk category focused summary.
# summariser = pipeline('summarization', model = 'facebook/bart-large-cnn')

# def pra_focused_summary(answer_text: str, k: int = 3, max_len: int = 100, min_len: int = 40) -> str:
#     top = retrieve_pra_context(answer_text, k=k)
#     context = make_context_block(top)

#     prompt = (
#         'Summarise the following banker answer for PRA stakeholders.\n'
#         'Focus on concrete details related to the most relevant risk categories below.\n\n'
#         'Relevant PRA risk categories:\n'
#         f'{context}\n\n'
#         'Banker answer:\n'
#         f'{answer_text}\n\n'
#         'Return 2 bullet points with a summary and then how it related to PRA risks'
#     )

#     out = summariser(prompt, max_length=max_len, min_length=min_len, do_sample=False)

#     return out[0]['summary_text']

Device set to use mps:0


In [ ]:
# Test on a sample.
banker_answers_df = jpm_2025_df[jpm_2025_df['role_normalised'] == 'banker'][['content']]
banker_answers_df_sample = banker_answers_df.head(25).copy()
banker_answers_df_sample['pra_summary'] = banker_answers_df_sample['content'].apply(lambda t: pra_focused_summary(t, k=2))
banker_answers_df_sample['pra_summary']

# **6. Evasion Scoring**